In [1]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json
import pprint
from datetime import datetime

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# Confirm list of databases
print(mongo.list_database_names())

['admin', 'autosaurus', 'chicago_bikes', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'mongodbVSCodePlaygroundDB', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# Assign the database to a variable name
db = mongo.chicago_bikes

In [5]:
# Review collections in database
print(db.list_collection_names())

['distinct_station_names', 'divvy_ridedata_merged', 'RouteDistance', 'divvy_rides_by_season', 'withStationName', 'weather_daily', 'Top10EndStations', 'Top10Routes', 'sig_prcp_yes', 'sig_prcp_no', 'withoutStationName', 'withLatLong', 'divvy_ridedata', 'divvy_rides_by_month', 'Top10StartStations']


In [6]:
# Assign weather data to a variable 
weather_daily = db['weather_daily']

In [7]:
# Count the number of documents in weather data 
print(weather_daily.count_documents({}))

365


In [8]:
# Review a document in the weather_daily collection 
print(db.weather_daily.find_one())

{'_id': ObjectId('6544a21516ac0221c1a8c57f'), 'date': '2022-01-01', 'cloud_cover': 90.0, 'precipitation': 0.18, 'min_temp': 33.22, 'max_temp': 42.1, 'morning_temp': 42.1, 'afternoon_temp': 38.43, 'evening_temp': 35.24, 'night_temp': 38.44, 'max_windspeed': 15.01, 'prcp_inches': 0.0070866, 'sig_prcp': 'no', 'avg_temp': 38.59}


In [9]:
# Define the aggregation pipeline to calculate monthly averages
pipeline = [
    {
        "$project": {
            "month": {"$month": {"$toDate": "$date"}},  # Convert "date" to Date type and then extract the month
            "min_temp": "$min_temp",
            "max_temp": "$max_temp",
            "precipitation": "$precipitation",  # Assuming "prcp_inches" is the field with precipitation in inches
        }
    },
    {
        "$group": {
            "_id": "$month",
            "avg_min_temp": {"$avg": "$min_temp"},
            "avg_max_temp": {"$avg": "$max_temp"},
            "avg_precipitation": {"$avg": "$precipitation"},
        }
    },
    {
        "$sort": {"_id": 1}
    }
]

In [10]:
# Execute the aggregation pipeline
result = list(weather_daily.aggregate(pipeline))

In [11]:
# Print the results
for month_data in result:
    month = month_data["_id"]
    avg_min_temp = month_data["avg_min_temp"]
    avg_max_temp = month_data["avg_max_temp"]
    avg_precipitation = month_data["avg_precipitation"]
    print(f"Month: {month}, Avg Min Temp: {avg_min_temp:.2f}°F, Avg Max Temp: {avg_max_temp:.2f}°F, Avg precipitation: {avg_precipitation} inches")

Month: 1, Avg Min Temp: 14.02°F, Avg Max Temp: 27.96°F, Avg precipitation: 1.315806451612903 inches
Month: 2, Avg Min Temp: 20.09°F, Avg Max Temp: 34.76°F, Avg precipitation: 2.6885714285714286 inches
Month: 3, Avg Min Temp: 33.26°F, Avg Max Temp: 50.77°F, Avg precipitation: 3.3270967741935484 inches
Month: 4, Avg Min Temp: 41.26°F, Avg Max Temp: 55.70°F, Avg precipitation: 2.396666666666667 inches
Month: 5, Avg Min Temp: 56.25°F, Avg Max Temp: 73.21°F, Avg precipitation: 2.105161290322581 inches
Month: 6, Avg Min Temp: 63.45°F, Avg Max Temp: 84.04°F, Avg precipitation: 1.3813333333333333 inches
Month: 7, Avg Min Temp: 67.41°F, Avg Max Temp: 84.28°F, Avg precipitation: 3.9932258064516133 inches
Month: 8, Avg Min Temp: 66.52°F, Avg Max Temp: 82.94°F, Avg precipitation: 1.326451612903226 inches
Month: 9, Avg Min Temp: 59.74°F, Avg Max Temp: 77.16°F, Avg precipitation: 0.6896666666666668 inches
Month: 10, Avg Min Temp: 46.69°F, Avg Max Temp: 63.98°F, Avg precipitation: 2.0316129032258066 

In [12]:
# Close the MongoDB connection
mongo.close()